# Apache Kafkaの環境を構築するためのノードを起動する

Apache Kafka の環境を構築するノードを AWS EC2インスタンスで起動します。

ここでは、以下のノードを起動することを想定しています。

* Apache Kafka Broker
  - 3ノード
* Kafka Connector
  - 1ノード

# パラメータの指定

## Vcp SDK

ノードの起動には Vcp SDK を利用します。VCコントローラのアクセストークンを用意してください。

次のセルを実行すると表示される入力枠にアクセストークンを入力してください。

In [ ]:
from getpass import getpass
access_token = getpass()

入力されたアクセストークンが正しいことを確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(access_token)

## VCノードに割り当てるリソース

### broker

Apache Kafka の broker を実行するVCノードに割り当てるリソース量を指定します。

VCノードとして起動するAWS EC2のインスタンスタイプを指定してください。

In [ ]:
broker_instance_type = 'm4.large'

VCノードに割り当てるディスクサイズ(GB)を指定してください。

In [ ]:
broker_disk_size = 64

VCノードに割り当てる IP アドレスを指定してください。

In [ ]:
broker_ip_addresses = [
    '172.30.2.10',
    '172.30.2.11',
    '172.30.2.12',
]

### Kafkaコネクタ

VCノードとして起動するAWS EC2のインスタンスタイプを指定してください。

In [ ]:
connector_instance_type = 'm4.large'

VCノードに割り当てるディスクサイズ(GB)を指定してください。

In [ ]:
connector_disk_size = 32

VCノードに割り当てる IP アドレスを指定してください。

In [ ]:
connector_ip_address = '172.30.2.20'

## UnitGroup名

この Notebook で起動するVCノードをグループとして扱う場合の名前を指定してください。

In [ ]:
ugroup_name = 'dp-kafka'

## SSHの公開鍵

起動したVCノードに登録する SSH の公開鍵のパスを指定してください。

In [ ]:
from pathlib import Path
ssh_publickey_path = Path('~/.ssh/id_rsa.pub').expanduser()

公開鍵とペアになる秘密鍵のパスを指定してください。

In [ ]:
ssh_privatekey_path = Path('~/.ssh/id_rsa').expanduser()

# VCノードの起動

## specの作成

In [ ]:
spec_b = vcp.get_spec("aws", "medium")
spec_b.image = 'vcp/base:1.3-centos'
spec_b.set_ssh_pubkey(ssh_publickey_path)
spec_b.instance_type = broker_instance_type
spec_b.volume_size = broker_disk_size
spec_b.num_nodes = len(broker_ip_addresses)
spec_b.ip_addresses = broker_ip_addresses
print(spec_b)

In [ ]:
spec_c = vcp.get_spec("aws", "medium")
spec_c.image = 'vcp/base:1.3-centos'
spec_c.set_ssh_pubkey(ssh_publickey_path)
spec_c.instance_type = connector_instance_type
spec_c.volume_size = connector_disk_size
spec_c.ip_addresses = [connector_ip_address]
print(spec_c)

## VCノードの起動

VCノードを起動します。

In [ ]:
ug = vcp.create_ugroup(ugroup_name)
ug.create_unit('broker', spec_b)
ug.create_unit('connector', spec_c)

VCノードの状態を確認してみます。

In [ ]:
ug.df_nodes()

# Ansibleの設定

VCノードをAnsibleのインベントリに登録します。

In [ ]:
inventory = Path('./hosts')
if inventory.exists():
    inventory.rename(inventory.with_suffix('.orig'))

with inventory.open(mode='w') as f:
    for unit in ug.find_units():
        f.write(f'[{unit.name}]\n')
        for idx, x in enumerate(unit.find_ip_addresses()):
            f.write(f'{unit.name}-{idx} ansible_host={x}\n')
        f.write(f'''
[{unit.name}:vars]
ansible_user=root
ansible_ssh_private_key_file={ssh_privatekey_path}

''')
    f.write(f'[{ugroup_name}:children]\n')
    for unit in ug.find_units():
        f.write(f'{unit.name}\n')
    
!cat {inventory}
if inventory.with_suffix('.orig').exists():
    try:
        !diff -u {inventory.with_suffix('.orig')} {inventory}
    except:
        pass

`~/.ssh/known_hosts`を更新します。

In [ ]:
for addr in ug.find_ip_addresses():
    !ssh-keygen -R {addr}
    !ssh-keyscan -H {addr} >> ~/.ssh/known_hosts

インベントリのパスを `ansible.cfg` に設定します。

In [ ]:
ansible_cfg = Path('./ansible.cfg')
if ansible_cfg.exists():
    ansible_cfg.rename(ansible_cfg.with_suffix('.orig'))

with ansible_cfg.open(mode='w') as f:
    f.write(f'''
[defaults]
inventory = {inventory.absolute()}
''')
    
!cat {ansible_cfg}
if ansible_cfg.with_suffix('.orig').exists():
    try:
        !diff -u {ansible_cfg.with_suffix('.orig')} {ansible_cfg}
    except:
        pass

疎通確認を行います。

In [ ]:
!ansible {ugroup_name} -m ping

# /etc/hosts の更新

Apache Kafkaはホスト名の解決が必要となるので、`/etc/hosts` に起動したVCノードのホスト名を登録します。

In [ ]:
hosts_dict = dict([
    (node.cloud_instance_address, f"{unit.name}-{idx}")
    for unit in ug.find_units()
    for idx, node in enumerate(unit.find_nodes())
])
etc_hosts = "\n".join([f"{k} {v}" for k, v in hosts_dict.items()])

!ansible {ugroup_name} -m shell -a 'warn=False \
    mkdir -p /tmp/{ugroup_name}; cp /etc/hosts /tmp/{ugroup_name}/'
!ansible {ugroup_name} -b -m blockinfile -a \
    'path=/tmp/{ugroup_name}/hosts block="{etc_hosts}"'
!ansible {ugroup_name} -b -a 'warn=False cp /tmp/{ugroup_name}/hosts /etc/hosts'
!ansible {ugroup_name} -b -a 'warn=False rm -rf /tmp/{ugroup_name}'

更新後の `/etc/hosts` の内容を確認します。

In [ ]:
!ansible {ugroup_name} -a 'cat /etc/hosts'

Notebook環境の `/etc/hosts` にも同様の内容を追加しておきます。

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path

with TemporaryDirectory() as work_dir:
    !cp /etc/hosts {work_dir}
    !ansible localhost -c local -b -m blockinfile -a \
        'path={work_dir}/hosts block="{etc_hosts}"'
    try:
        !diff -u /etc/hosts {work_dir}/hosts
    except:
        pass
    !sudo cp {work_dir}/hosts /etc/hosts

# ツール類のインストール

## 追加パッケージ

環境構築の際に必要となるパッケージをインストールします。

In [ ]:
!ansible {ugroup_name} -b -m yum -a 'name=rsync,openssh-clients update_cache=yes'

## docker-compose

In [ ]:
!ansible {ugroup_name} -b -m shell -b -a 'warn=False \
    curl -L "https://github.com/docker/compose/releases/download/1.23.2/docker-compose-$(uname -s)-$(uname -m)" \
    -o /usr/local/bin/docker-compose'
!ansible {ugroup_name} -b -a 'warn=False chmod +x /usr/local/bin/docker-compose'